In [ ]:
import glob
import pandas as pd
import uproot
import uproot_methods
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LightSource
import matplotlib.colors as mcolors

%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patheffects as PathEffects
import matplotlib.ticker as mticker

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import interpolate

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import itertools

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

dir = r"../PYTHON_graphs/OUTPUTS/standard_graphs/"
dirN2 = r"../PYTHON_graphs/OUTPUTS/N2_calibration/"

allfiles = glob.glob(r"../PYTHON_graphs/DATA/Standard_graphs/*.root")

cmap_viridis = mpl.cm.get_cmap('viridis')

In [ ]:
allfiles

In [ ]:
for el in allfiles:
    if "elec_linearity" in el:
        elec_lin = uproot.open(el)
        x_el_lin,y_el_lin,counts_el_lin=fugi.import_TH2Dgeneric(elec_lin,elec_lin.keys()[0])
    elif "pipico_momcon_shrt" in el:
        pipico_n2 = uproot.open(el)
    elif "recoil_sum_w" in el:
        recoil_sum_w = uproot.open(el)
        w_layer_rec,tsum_rec,counts_rec_sum=fugi.import_TH2Dgeneric(recoil_sum_w,recoil_sum_w.keys()[0])
    elif "rec_xy" in el:
        recoil_xy = uproot.open(el)
        x_rec,y_rec,counts_rec_xy=fugi.import_TH2Dgeneric(recoil_xy,recoil_xy.keys()[0])
    elif "pipico_momcon" in el:
        pipico_n2_momcon = uproot.open(el)
        tof1,tof2,counts_pipico=fugi.import_TH2Dgeneric(pipico_n2_momcon,pipico_n2_momcon.keys()[0])
    elif "elec_sum_w" in el:
        elec_sum_w = uproot.open(el)
        w_layer,tsum,counts_el_sum=fugi.import_TH2Dgeneric(elec_sum_w,elec_sum_w.keys()[0])
    elif "elec_xy" in el:
        elec_xy = uproot.open(el)
        x_el,y_el,counts_el_xy=fugi.import_TH2Dgeneric(elec_xy,elec_xy.keys()[0])
    elif "TFMOX_pipico_short" in el:
        tfmox_pipico = uproot.open(el)
        tmox_tof1,tmox_tof2,counts_pipico_tmox=fugi.import_TH2Dgeneric(tfmox_pipico,tfmox_pipico.keys()[0])
    elif "PIPICO_CH9" in el:
        tfmox_pipico_ch9 = uproot.open(el)
        tof9_1,tof9_2,couts_pipico_9=fugi.import_TH2Dgeneric(tfmox_pipico_ch9,tfmox_pipico_ch9.keys()[0])

In [ ]:
IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/","ID4_mol_e0_valid/EN_gate/"]
ID=str(IDs[3]).split("_")[0]

# dirs = ["MFPADs_multinew/","MFPADs_multinew_phi/"]

fileSCR_12 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
path12 = "angular_distr_el/CH12/"
loc12=path12+IDs[1] #ID 1 is the only one possible

fileSCR_11_11 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH11_550eV_CR_Knov_test.root")
fileSCR_11_8 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_10800-2350ns_multiCH11.root")
fileSCR_11_6 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_9600-3700ns_multiCH11.root")
fileSCR_11_4 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_10800-2350ns_COMBI_multiCH11.root")
fileSCR_11_3 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_10800-2350ns_multiCH11.root")
path11 = "angular_distr_el/CH11/"
loc11=path11+IDs[3] #ID 0 for all channels together

fileSCR_9 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_CH9_550eV_CR_Knov.root")
path9 = "angular_distr_el/CH9/"
loc9=path9+IDs[3] #ID 0 for all channels together

fileSCR_14 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CR_ALLCH_diatomic.root")
path14 = "angular_distr_el/CH14/"
loc14=path14+IDs[0]

pz,px,counts_newton11=fugi.import_TH2Dgeneric(fileSCR_11_11,loc11+"/Newton plot")

en,KER=fugi.import_TH1Dgeneric(fileSCR_14,"FMOx_m43-69/mol_ion/KER")
ctheta,cKER,counts_KER=fugi.import_TH2Dgeneric(fileSCR_14,"FMOx_m43-69/mol_ion/ctheta vs KER")

en11,KER11=fugi.import_TH1Dgeneric(fileSCR_11_11,"FMOx_m30-69/mol_ion/KER")
ctheta11,cKER11,counts_KER11=fugi.import_TH2Dgeneric(fileSCR_11_11,"FMOx_m30-69/mol_ion/ctheta vs KER")
px_MF,py_MF,pz_MF,counts=fugi.import_TH3Dgeneric(fileSCR_11_11,"angular_distr_el/CH11/ID_ALL_mol_e0_valid/EN_gate/mol_orient_GATED/Ion Molframe cnorm")
px_MF_l,py_MF_l,pz_MF_l,counts_l=fugi.import_TH3Dgeneric(fileSCR_11_11,"angular_distr_el/CH11/ID_ALL_mol_e0_valid/EN_gate/mol_orient_GATED/Ion Molframe coord")

prelx,prelz,counts_prel=fugi.import_TH2Dgeneric(fileSCR_11_11,"test/p_relx vs p_relz")

pz,px,counts_newton9=fugi.import_TH2Dgeneric(fileSCR_9,loc9+"/Newton plot")
# check_ion[2]_neutral_xy
# KER, p_sumx vs p_sumz, p_relx vs p_relz

tof14_1,tof14_2,couts_pipico_14=fugi.import_TH2Dgeneric(fileSCR_14,loc14+"/PIPICO")
tof11_1,tof11_2,couts_pipico_11=fugi.import_TH2Dgeneric(fileSCR_11_11,"test/PIPICO")

In [ ]:
newton12_all=[]; newton11_11_all=[]; newton11_8_all=[]; newton11_6_all=[]; newton11_6_all=[]; newton11_4_all=[]; newton11_3_all=[]; newton9_all=[]
for i,el in enumerate(IDs):
    if i == 1:
        loc12=path12+el
        dummy1,dummy2,plot=fugi.import_TH2Dgeneric(fileSCR_12,loc12+"/Newton plot")
        newton12_all.append(plot)
    else:
        loc11=path11+el        
        dummy1,dummy2,plot11=fugi.import_TH2Dgeneric(fileSCR_11_11,loc11+"/Newton plot")
        dummy1,dummy2,plot8=fugi.import_TH2Dgeneric(fileSCR_11_8,loc11+"/Newton plot")
        dummy1,dummy2,plot6=fugi.import_TH2Dgeneric(fileSCR_11_6,loc11+"/Newton plot")
        dummy1,dummy2,plot4=fugi.import_TH2Dgeneric(fileSCR_11_4,loc11+"/Newton plot")
        dummy1,dummy2,plot3=fugi.import_TH2Dgeneric(fileSCR_11_3,loc11+"/Newton plot")
        newton11_11_all.append(plot11)
        newton11_8_all.append(plot8)
        newton11_6_all.append(plot6)
        newton11_4_all.append(plot4)
        newton11_3_all.append(plot3)
    if i <=3:
        loc9=path9+el
        dummy1,dummy2,plot=fugi.import_TH2Dgeneric(fileSCR_9,loc9+"/Newton plot")
        newton9_all.append(plot)

In [ ]:
MFPAD_RMOx_CL=pd.read_table(r'../PYTHON_graphs/DATA/MOx_Kilian/RMoxCLMFPAD.dat',names=['phi','ctheta','light_dir','counts'],sep='\t',skiprows=1)
MFPAD_SMOx_CL=pd.read_table(r'../PYTHON_graphs/DATA/MOx_Kilian/SMoxCLMFPAD.dat',names=['phi','ctheta','light_dir','counts'],sep='\t',skiprows=1)
MFPAD_RMOx_CR=pd.read_table(r'../PYTHON_graphs/DATA/MOx_Kilian/RMoxCRMFPADmirror.dat',names=['phi','ctheta','light_dir', 'counts'],sep='\t',skiprows=1)
MFPAD_SMOx_CR=pd.read_table(r'../PYTHON_graphs/DATA/MOx_Kilian/SMoxCRMFPADmirror.dat',names=['phi','ctheta','light_dir', 'counts'],sep='\t',skiprows=1)

MFPAD_RMOx_CL.sort_values("light_dir", inplace=True); MFPAD_SMOx_CL.sort_values("light_dir", inplace=True)
MFPAD_RMOx_CR.sort_values("light_dir", inplace=True); MFPAD_SMOx_CR.sort_values("light_dir", inplace=True)

SCLtemp=[]; RCLtemp=[]; SCRtemp=[]; RCRtemp=[];
Stemp=np.empty([6,12,54*24]) #MFPAD
Rtemp=np.empty([6,12,54*24]) #MFPAD
temp_sum=np.empty([6,12,54*24]) #MFPAD
temp_PECD_S=np.empty([6,12,54*24])
temp_PECD=np.empty([6,12,54*24])

for i in np.arange(72):
    RCLtemp.append(MFPAD_RMOx_CL.loc[MFPAD_RMOx_CL["light_dir"] == i].sort_values(["phi","ctheta"])["counts"])
    SCLtemp.append(MFPAD_SMOx_CL.loc[MFPAD_SMOx_CL["light_dir"] == i].sort_values(["phi","ctheta"])["counts"])
    RCRtemp.append(MFPAD_RMOx_CR.loc[MFPAD_RMOx_CR["light_dir"] == i].sort_values(["phi","ctheta"])["counts"])
    SCRtemp.append(MFPAD_SMOx_CR.loc[MFPAD_SMOx_CR["light_dir"] == i].sort_values(["phi","ctheta"])["counts"])
    if i == 0:
        cthea_MOx_full=MFPAD_SMOx_CL.loc[MFPAD_SMOx_CL["light_dir"] == i]["ctheta"].to_numpy()
        phi_MOx_full=MFPAD_SMOx_CL.loc[MFPAD_SMOx_CL["light_dir"] == i]["phi"].to_numpy()

cthea_MOx=np.unique(cthea_MOx_full)
phi_MOx=np.unique(phi_MOx_full)

RCLMOx_MFPAD=np.array(RCLtemp).reshape(72,54,24)
RCLMOx_MFPAD_tot=np.sum(RCLMOx_MFPAD, axis=0)

SCLMOx_MFPAD=np.array(SCLtemp).reshape(72,54,24)
SCLMOx_MFPAD_tot=np.sum(SCLMOx_MFPAD, axis=0)

RCRMOx_MFPAD=np.array(RCRtemp).reshape(72,54,24)
RCRMOx_MFPAD_tot=np.sum(RCRMOx_MFPAD, axis=0)

SCRMOx_MFPAD=np.array(SCRtemp).reshape(72,54,24)
SCRMOx_MFPAD_tot=np.sum(SCRMOx_MFPAD, axis=0)

#NOTE direct sum of helicities, BUT CR is mirrored
for i in range(6):
    for j in range(12):
        Rtemp[i,j]=np.add(RCLMOx_MFPAD.reshape(6,12,-1)[i,j],RCRMOx_MFPAD.reshape(6,12,-1)[i,j])
        Stemp[i,j]=np.add(SCLMOx_MFPAD.reshape(6,12,-1)[i,j],SCRMOx_MFPAD.reshape(6,12,-1)[i,j])

RMOx_MFPAD=np.array(Rtemp).reshape(72,54,24)
RMOx_MFPAD_flip=np.flip(RMOx_MFPAD,axis=2) #flip cos(theta) len=24
RMOx_MFPAD_flip_tot=np.sum(RMOx_MFPAD, axis=0)

SMOx_MFPAD=np.array(Stemp).reshape(72,54,24)
SMOx_MFPAD=np.array(Stemp).reshape(72,54,24)
SMOx_MFPAD_flip_tot=np.sum(SMOx_MFPAD, axis=0)

for i in range(6):
    for j in range(12):
          temp_sum[i,j]=np.add(SMOx_MFPAD.reshape(6,12,-1)[i,j],RMOx_MFPAD_flip.reshape(6,12,-1)[5-i,j])
    
totMOx_MFPAD=np.array(temp_sum).reshape(72,54,24)
totMOx_MFPAD_tot=np.sum(totMOx_MFPAD, axis=0)

for i in range(6):
    for j in range(12):
        if j>5:
            temp_PECD_S[i,j]=np.divide(np.subtract(SMOx_MFPAD.reshape(6,12,-1)[i,j],SMOx_MFPAD.reshape(6,12,-1)[5-i,j-6]),np.add(SMOx_MFPAD.reshape(6,12,-1)[i,j],SMOx_MFPAD.reshape(6,12,-1)[5-i,j-6]))
            temp_PECD[i,j]=np.divide(np.subtract(totMOx_MFPAD.reshape(6,12,-1)[i,j],totMOx_MFPAD.reshape(6,12,-1)[5-i,j-6]),np.add(totMOx_MFPAD.reshape(6,12,-1)[i,j],totMOx_MFPAD.reshape(6,12,-1)[5-i,j-6]))
        else:
            temp_PECD_S[i,j]=np.divide(np.subtract(SMOx_MFPAD.reshape(6,12,-1)[i,j],SMOx_MFPAD.reshape(6,12,-1)[5-i,6+j]),np.add(SMOx_MFPAD.reshape(6,12,-1)[i,j],SMOx_MFPAD.reshape(6,12,-1)[5-i,6+j]))
            temp_PECD[i,j]=np.divide(np.subtract(totMOx_MFPAD.reshape(6,12,-1)[i,j],totMOx_MFPAD.reshape(6,12,-1)[5-i,6+j]),np.add(totMOx_MFPAD.reshape(6,12,-1)[i,j],totMOx_MFPAD.reshape(6,12,-1)[5-i,6+j]))


PECD_Mox_S=np.array(temp_PECD_S).reshape(72,54,24)
PECD_Mox_tot=np.array(temp_PECD_S).reshape(72,54,24)

In [ ]:
"""
The original PBD id centered aroud the #2 C and the #3 and #2 C lay on a line: the molecule appears to be rotated of about 30 deg with
respect to Blender, which implies shifting in phi angle 
"""
def listOfTuples(l1, l2):
    return list(map(lambda x, y:(x,y), l1, l2))

def listOfTuples3D(l1, l2, l3):
    return list(map(lambda x, y, z:(x,y,z), l1, l2, l3))

PDB_S_coord_raw=pd.read_table(r'../PYTHON_graphs/MOL_structures/S-TMOX.pdb',header=None,delim_whitespace=True,nrows=10,skiprows=2)
df_S_PDB=PDB_S_coord_raw.drop(PDB_S_coord_raw.columns[[0,1,3,4,8,9,10]], axis=1)
df_S_PDB.columns = ['atom', 'x', 'y', 'z']
df_R_PDB=df_S_PDB.copy()

edge_list=[(0,6),(1,6),(2,6),(3,4),(3,5),(4,3),(4,5),(4,6),(4,7),(5,3),(5,4),(5,8),(5,9),(6,0),(6,1),(6,2),(6,4),(7,4),(8,5),(9,5)]

a=-125.; b=0.; g=0.

# df_R_PDB['x'] = 0.664 + df_R_PDB['x'] #shifting the origin of the molecule
# df_R_PDB['y'] = 1. + df_R_PDB['y'] #shifting the origin of the molecule
# df_R_PDB['z'] = +0.3 + df_R_PDB['z'] #shifting the origin of the molecule
df_R_PDB['z'] = -1 * df_R_PDB['z'] #mirroring the S enantiomer

##################### R enationer ########################
xyzm = np.stack((df_R_PDB['x'], df_R_PDB['y'], df_R_PDB['z']))
x_rot_R, y_rot_R, z_rot_R = np.einsum('ik, kj -> ij', fugi.rot3d(a*np.pi/180,b*np.pi/180,g*np.pi/180,convention=1).T, xyzm)

r_R=[];ctheta_PDB_R=[];phi_PDB_R=[]; r_R_rot=[];ctheta_PDB_R_rot=[];phi_PDB_R_rot=[];
for index, row in df_R_PDB.iterrows():
    r_R.append(np.sqrt(row['x']**2+row['y']**2+row['z']**2))
    ctheta_PDB_R.append(row['z']/r_R[index])
    phi_PDB_R.append(np.arctan2(row['y'],row['x'])*180./np.pi)

    r_R_rot.append(np.sqrt(x_rot_R[index]**2+y_rot_R[index]**2+z_rot_R[index]**2))
    ctheta_PDB_R_rot.append(z_rot_R[index]/r_R_rot[index])
    phi_PDB_R_rot.append(np.arctan2(y_rot_R[index],x_rot_R[index])*180./np.pi)

df_R_PDB.insert(4, "x_rot", x_rot_R, True); df_R_PDB.insert(5, "y_rot", y_rot_R, True); df_R_PDB.insert(6, "z_rot", z_rot_R, True);
df_R_PDB.insert(7, "phi", phi_PDB_R, True); df_R_PDB.insert(8, "ctheta", ctheta_PDB_R, True);
df_R_PDB.insert(9, "phi_rot", phi_PDB_R_rot, True); df_R_PDB.insert(10, "ctheta_rot", ctheta_PDB_R_rot, True)

xyzm_rot = np.stack((df_R_PDB['x_rot'], df_R_PDB['y_rot'], df_R_PDB['z_rot']))

#we  need to create lists that contain the starting and ending coordinates of each edge.
x_edges_R=[]; y_edges_R=[]; z_edges_R=[]
x_edges_rot_R=[]; y_edges_rot_R=[]; z_edges_rot_R=[]

#need to fill these with all of the coordiates
for edge in edge_list:
    x_coords_rot = [xyzm_rot.T[edge[0]][0],xyzm_rot.T[edge[1]][0],None]
    x_edges_rot_R += x_coords_rot
    x_coords = [xyzm.T[edge[0]][0],xyzm.T[edge[1]][0],None]
    x_edges_R += x_coords

    y_coords_rot = [xyzm_rot.T[edge[0]][1],xyzm_rot.T[edge[1]][1],None]
    y_edges_rot_R += y_coords_rot
    y_coords = [xyzm.T[edge[0]][1],xyzm.T[edge[1]][1],None]
    y_edges_R += y_coords

    z_coords_rot = [xyzm_rot.T[edge[0]][2],xyzm_rot.T[edge[1]][2],None]
    z_edges_rot_R += z_coords_rot
    z_coords = [xyzm.T[edge[0]][2],xyzm.T[edge[1]][2],None]
    z_edges_R += z_coords


####################### S enationer ######################
xyzm = np.stack((df_S_PDB['x'], df_S_PDB['y'], df_S_PDB['z']))
x_rot_S, y_rot_S, z_rot_S = np.einsum('ik, kj -> ij', fugi.rot3d(a*np.pi/180,b*np.pi/180,g*np.pi/180,convention=1).T, xyzm)

r_S=[];ctheta_PDB_S=[];phi_PDB_S=[];r_S_rot=[];ctheta_PDB_S_rot=[];phi_PDB_S_rot=[];
for index, row in df_S_PDB.iterrows():
    r_S.append(np.sqrt(row['x']**2+row['y']**2+row['z']**2))
    ctheta_PDB_S.append(row['z']/r_S[index])
    phi_PDB_S.append(np.arctan2(row['y'],row['x'])*180./np.pi)

    r_S_rot.append(np.sqrt(x_rot_S[index]**2+y_rot_S[index]**2+z_rot_S[index]**2))
    ctheta_PDB_S_rot.append(z_rot_S[index]/r_S_rot[index])
    phi_PDB_S_rot.append(np.arctan2(y_rot_S[index],x_rot_S[index])*180./np.pi)

df_S_PDB.insert(4, "x_rot", x_rot_S, True); df_S_PDB.insert(5, "y_rot", y_rot_S, True); df_S_PDB.insert(6, "z_rot", z_rot_S, True);
df_S_PDB.insert(7, "phi", phi_PDB_S, True); df_S_PDB.insert(8, "ctheta", ctheta_PDB_S, True);
df_S_PDB.insert(9, "phi_rot", phi_PDB_S_rot, True); df_S_PDB.insert(10, "ctheta_rot", ctheta_PDB_S_rot, True)

xyzm_rot = np.stack((df_S_PDB['x_rot'], df_S_PDB['y_rot'], df_S_PDB['z_rot']))

x_edges_S=[]; y_edges_S=[]; z_edges_S=[]
x_edges_rot_S=[]; y_edges_rot_S=[]; z_edges_rot_S=[]

for edge in edge_list:
    x_coords_rot = [xyzm_rot.T[edge[0]][0],xyzm_rot.T[edge[1]][0],None]
    x_edges_rot_S += x_coords_rot
    x_coords = [xyzm.T[edge[0]][0],xyzm.T[edge[1]][0],None]
    x_edges_S += x_coords

    y_coords_rot = [xyzm_rot.T[edge[0]][1],xyzm_rot.T[edge[1]][1],None]
    y_edges_rot_S += y_coords_rot
    y_coords = [xyzm.T[edge[0]][1],xyzm.T[edge[1]][1],None]
    y_edges_S += y_coords

    z_coords_rot = [xyzm_rot.T[edge[0]][2],xyzm_rot.T[edge[1]][2],None]
    z_edges_rot_S += z_coords_rot
    z_coords = [xyzm.T[edge[0]][2],xyzm.T[edge[1]][2],None]
    z_edges_S += z_coords

In [ ]:
df_R_PDB

In [ ]:
trace_nodes = go.Scatter3d(x=df_R_PDB['x_rot'], y=df_R_PDB['y_rot'], z=df_R_PDB['z_rot'],
                        text=df_R_PDB['atom'], mode='markers', marker=dict( size=15, opacity=0.8),name='atoms')

trace_edges_rot = go.Scatter3d(x=x_edges_rot_R, y=y_edges_rot_R, z=z_edges_rot_R,
                        mode='lines', line=dict(color='black', width=6), name='bonds rot',hoverinfo='none')

trace_edges = go.Scatter3d(x=x_edges_R,y=y_edges_R,z=z_edges_R,
                        mode='lines',line=dict(color='red', width=2),name='original',hoverinfo='none')

origin = go.Scatter3d(x=[0.],y=[0.],z=[0.],
                     mode='markers', marker=dict(color='red', size=10, opacity=1),name="origin")

x= np.linspace(-2, 2, 20); y= np.linspace(-2, 2, 20); z= 0*np.ones((20,20))
mycolorscale = [[0, '#aa9ce2'],[1, '#aa9ce2']]

surf = go.Surface(x=x, y=y, z=z, colorscale=mycolorscale, opacity=0.4, showscale=False)

data = [trace_nodes, trace_edges_rot, trace_edges, origin, surf]
fig = go.Figure(data=data)

fig.update_layout(width=900, height=800)
fig.show()

In [ ]:
fig,ax=plt.subplots(1,3, figsize=(35,10))
xy_R = listOfTuples(df_R_PDB["phi"], df_R_PDB["ctheta"])
xy_S = listOfTuples(df_S_PDB["phi"], df_S_PDB["ctheta"])
xy_rot_R = listOfTuples(df_R_PDB["phi_rot"], df_R_PDB["ctheta_rot"])
xy_rot_S = listOfTuples(df_S_PDB["phi_rot"], df_S_PDB["ctheta_rot"])
aspect=(df_R_PDB["phi"].max()/df_R_PDB["ctheta"].max())

ax[0].plot(df_R_PDB["phi"], df_R_PDB["ctheta"], "o")
for index, row in df_R_PDB.iterrows():
    ax[0].annotate(row['atom'], xy=xy_R[index], textcoords='data')

ax[0].set_title("R enant atom position")
ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_xlim([-175,175])
ax[0].set_ylabel('cos\u03D1 [adm]')
ax[0].set_ylim([-0.944,0.944])
# ax[0].set_aspect(aspect)
ax[0].set_aspect(aspect, adjustable='box')
ax[0].grid(linestyle = '-', linewidth = 2 , alpha = 0.6)

ax[1].plot(df_R_PDB["phi_rot"], df_R_PDB["ctheta_rot"], "o")
for index, row in df_R_PDB.iterrows():
    ax[1].annotate(row['atom'], xy=xy_rot_R[index], textcoords='data')

ax[1].set_title("R enant atom position ROT")
ax[1].set_xlabel('\u03C6 [DEG]')
ax[1].set_xlim([-175,175])
ax[1].set_ylabel('cos\u03D1 [adm]')
ax[1].set_ylim([-0.944,0.944])
# ax[1].set_aspect(aspect)
ax[1].set_aspect(aspect, adjustable='box')
ax[1].grid(linestyle = '-', linewidth = 2 , alpha = 0.6)

ax[2].plot(df_S_PDB["phi_rot"], df_S_PDB["ctheta_rot"], "o")
for index, row in df_S_PDB.iterrows():
    ax[2].annotate(row['atom'], xy=xy_rot_S[index], textcoords='data')

ax[2].set_title("S enant atom position ROT")
ax[2].set_xlabel('\u03C6 [DEG]')
ax[2].set_xlim([-175,175])
ax[2].set_ylabel('cos\u03D1 [adm]')
ax[2].set_ylim([-0.944,0.944])
# ax[2].set_aspect(aspect)
ax[2].set_aspect(aspect, adjustable='box')
ax[2].grid(linestyle = '-', linewidth = 2 , alpha = 0.6)

fig.savefig(dir+"atom_position_sphericalcoords.png", dpi=300, transparent=True, bbox_inches='tight')
fig.savefig(dir+"atom_position_sphericalcoords.svg", dpi=300, transparent=True, bbox_inches='tight')

In [ ]:
#just a test for PECD as in the diatomic case: sum over the phi_ph and sum over the phi_elec
#the problem is the mismatch between the integration over phi_ph and the definition of the Euler angle beta
testStot=fugi.normalise_with_err(np.sum((np.sum(SMOx_MFPAD.reshape(6,12,-1),axis=1)).reshape(6,54,24),axis=1),normtype=2)
testRtot=fugi.normalise_with_err(np.sum((np.sum(RMOx_MFPAD.reshape(6,12,-1),axis=1)).reshape(6,54,24),axis=1),normtype=2)

PECD_test = np.nan_to_num(np.divide(np.subtract(testStot,testRtot),np.add(testStot,testRtot))) #TEST

print(PECD_test.shape)

In [ ]:
beta_photon = np.linspace(-180,180,6)

fig, ax= plt.subplots(1, figsize=(12, 10))
cs,ax=fugi.plot_interpolation(beta_photon,cthea_MOx,(PECD_test*100).T, ax=ax, cmap=cmap_temp, xstep=1, ystep=0.01, n=20, gaussian=7.)
ax.set_title("integrated PECD S-R MOx @ 11.5 eV")

aspect=(beta_photon.max()/cthea_MOx.max())
ax.set_xlabel('\u03B2 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.ax.set_ylabel('PECD %')
ax.set_aspect(aspect)

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes,axis=0).ravel()): #along lines starting from the bottom left
    if True:
        cs,ax=fugi.plot_interpolation(phi_MOx,cthea_MOx,(PECD_Mox_tot[i]*100).T, ax=ax, cmap=cmap_temp, limits=[-40.,40.], kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
        ax.tick_params(labelsize=12)
    
    if i == 48:
        ax.spines[:].set_color('blue')
        ax.spines[:].set_linewidth(5)
    elif i == 18:
        ax.spines[:].set_color('red')
        ax.spines[:].set_linewidth(5)

cbar = fig.colorbar(cs, ax=axes.ravel().tolist(), ticks=mticker.MultipleLocator(10), anchor=(1.5,1), pad=-2.5)
cbar.ax.set_ylabel('PECD %')
fugi.overlaygraph(fig)
fig.savefig(dir+"S-MOx_72PECD_tot.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row')
#NOTE for cos ordered the scannig order of axis has to be np.flip(axes.T,axis=1).ravel()
for i,ax in enumerate(np.flip(axes,axis=0).ravel()):
    cs,ax=fugi.plot_interpolation(phi_MOx,cthea_MOx,totMOx_MFPAD[i].T, ax=ax, kind="linear", xstep=5, ystep=0.05, n=15, gaussian=1.2)
    ax.tick_params(labelsize=12)
    
    if i == 48:
        ax.spines[:].set_color('blue')
        ax.spines[:].set_linewidth(5)
    elif i == 18:
        ax.spines[:].set_color('red')
        ax.spines[:].set_linewidth(5)

fugi.overlaycbar(fig,cs,axes,MFPAD=True)
fugi.overlaygraph(fig)
fig.savefig(dir+"S-MOx_72MFPAD_sum.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs,ax=fugi.plot_interpolation(phi_MOx,cthea_MOx,totMOx_MFPAD_tot.T, ax=ax, xstep=1, ystep=0.01, n=20, gaussian=7.)
ax.set_title("S-MOx MFPAD at 11.5 eV")

aspect=(phi_MOx.max()/cthea_MOx.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig(dir+"S-MOx_MFPAD_tot.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
# cs=ax.contourf(phiM,cosM,MFPAD_tot_sum.T, cmap=cmap_temp, levels=15)
cs,ax=fugi.plot_interpolation(phi_MOx,cthea_MOx,RMOx_MFPAD_flip_tot.T, ax=ax, xstep=1, ystep=0.01, n=20, gaussian=7.)
ax.set_title("R-MOx MFPAD at 11.5 eV")

aspect=(phi_MOx.max()/cthea_MOx.max())
ax.set_xlabel('\u03C6 [DEG]')
ax.set_ylabel('cos\u03D1 [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalised counts')
ax.set_aspect(aspect)

fig.savefig(dir+"R-MOx_MFPAD.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))
counts_el_sum_red=np.ma.masked_array(counts_el_sum,counts_el_sum<3)

# cs=ax.contourf(w_layer,tsum,counts_el_sum_red.T, levels=40)
cs=ax.pcolormesh(w_layer,tsum,counts_el_sum_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('w-layer [ns]')
ax.set_ylabel('time sum [ns]')
ax.set_ylim(-1,1)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([counts_el_sum_red.min(),counts_el_sum_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"el_sum_wlayer.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
# d = np.linspace(-0.5, 0.5, 50)
counts_el_sum_time=np.sum(counts_el_sum,axis=0)
counts_el_sum_time_red=np.ma.masked_array(counts_el_sum_time,counts_el_sum_time<500).compressed()
tsum_red=np.ma.masked_array(tsum,counts_el_sum_time<500).compressed()

mean = sum(tsum_red*counts_el_sum_time_red)/sum(counts_el_sum_time_red)
sigma = np.sqrt(sum(counts_el_sum_time_red*(tsum_red-mean)**2)/sum(counts_el_sum_time_red))

def gaus_fit(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

popt,pcov = curve_fit(gaus_fit,tsum_red,counts_el_sum_time_red,p0=[max(counts_el_sum_time_red), mean, sigma])

FWHM=2*np.sqrt(2*np.log(2))*sigma
fit_info = "\u00B5 = %.2f ns \n$\u03c3_{std}$ = %.2f \nFWHM = %.2f ps" % (mean, sigma, FWHM*10**3)
# fit_info = (f"Gauss fit: \u00B5 = {mu:+.2f}, \u00b2 std = {std:+.2f}")

fig,ax= plt.subplots(1, figsize=(12, 10))
ax.set_xlabel('time sum [ns]')
ax.set_ylabel('counts')
ax.set_xlim([-0.8, 1])
ax.ticklabel_format(axis="y",style="sci",scilimits=(0,1))

ax.set_box_aspect(1)

ax.plot(tsum_red,counts_el_sum_time_red, color=cmap_viridis(0),alpha = 0.6, linewidth=3)
ax.plot(tsum_red,gaus_fit(tsum_red,*popt), color=cmap_viridis(0.3), ls="--",linewidth=2, label="Gaussian fit")
ax.plot([-FWHM/2*0.45,FWHM/2*1.05], [counts_el_sum_time_red.max()/2,counts_el_sum_time_red.max()/2], color=cmap_viridis(0.3), linewidth=2., ls='--', alpha=0.5)
plt.plot([], [], ' ', label=fit_info)

ax.legend(loc="best",fontsize="small")
ax.grid(linestyle = '-', linewidth = 2 , alpha = 0.6)

fig.savefig(dir+"el_sum_wlayer_proj.png", dpi=300, transparent=False, bbox_inches='tight')
fig.savefig(dir+"el_sum_wlayer_proj.svg", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))
counts_rec_sum_red=np.ma.masked_array(counts_rec_sum,counts_rec_sum<10)

# cs=ax.contourf(w_layer,tsum,counts_el_sum_red.T, levels=40)
cs=ax.pcolormesh(w_layer_rec,tsum_rec,counts_rec_sum_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('w-layer [ns]')
ax.set_ylabel('time sum [ns]')
ax.set_ylim(-2,2)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([counts_el_sum_red.min(),counts_el_sum_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"rec_sum_wlayer.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
# d = np.linspace(-0.5, 0.5, 50)
counts_rec_sum_time=np.sum(counts_rec_sum,axis=0)

counts_rec_sum_time_red1=np.ma.masked_array(counts_rec_sum_time,counts_rec_sum_time<500)
counts_rec_sum_time_red=np.ma.masked_array(counts_rec_sum_time,tsum_rec>0.4).compressed()
tsum_rec_red1=np.ma.masked_array(tsum_rec,counts_rec_sum_time<500)
tsum_rec_red=np.ma.masked_array(tsum_rec,tsum_rec>0.4).compressed()

mean = sum(tsum_rec_red*counts_rec_sum_time_red)/sum(counts_rec_sum_time_red)
sigma = np.sqrt(sum(counts_rec_sum_time_red*(tsum_rec_red-mean)**2)/sum(counts_rec_sum_time_red))

def gaus_fit(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

popt,pcov = curve_fit(gaus_fit,tsum_rec_red,counts_rec_sum_time_red,p0=[max(counts_rec_sum_time_red), mean, sigma])

FWHM=2*np.sqrt(2*np.log(2))*sigma
fit_info = "\u00B5 = %.2f ns \n$\u03c3_{std}$ = %.2f \nFWHM = %.2f ps" % (mean, sigma, FWHM*10**3)
# fit_info = (f"Gauss fit: \u00B5 = {mu:+.2f}, \u00b2 std = {std:+.2f}")

fig,ax= plt.subplots(1, figsize=(12, 10))
ax.set_xlabel('time sum [ns]')
ax.set_ylabel('counts')
ax.set_xlim([-0.8, 1])
ax.ticklabel_format(axis="y",style="sci",scilimits=(0,1))

ax.set_box_aspect(1)

ax.plot(tsum_rec_red,counts_rec_sum_time_red, color=cmap_viridis(0),alpha = 0.6, linewidth=3)
ax.plot(tsum_rec_red,gaus_fit(tsum_rec_red,*popt), color=cmap_viridis(0.3), ls="--",linewidth=2, label="Gaussian fit")
ax.plot([-FWHM/2*0.45,FWHM/2*1.05], [counts_rec_sum_time_red.max()/2,counts_rec_sum_time_red.max()/2], color=cmap_viridis(0.3), linewidth=2., ls='--', alpha=0.5)
plt.plot([], [], ' ', label=fit_info)

ax.legend(loc="best",fontsize="small")
ax.grid()


plt.plot()
fig.savefig(dir+"rec_sum_wlayer_proj.png", dpi=300, transparent=False, bbox_inches='tight')
fig.savefig(dir+"rec_sum_wlayer_proj.svg", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
xy_el=np.array(list(itertools.product(x_el, y_el)))
X_el=np.array([col[0] for col in xy_el])
Y_el=np.array([col[1] for col in xy_el])

xy_rec=np.array(list(itertools.product(x_rec, y_rec)))
X_rec=np.array([col[0] for col in xy_rec])
Y_rec=np.array([col[1] for col in xy_rec])

In [ ]:
#3D MFPAD
counts_el_xy_new=np.ma.masked_where(np.sqrt(X_el**2+Y_el**2)<25,counts_el_xy.reshape(-1)).reshape(len(x_el),len(y_el))
counts_el_xy_new_red=np.ma.masked_array(counts_el_xy_new, counts_el_xy_new<=5)
norm=mcolors.LogNorm(counts_el_xy_new_red.min(),counts_el_xy_new_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
# cs=ax.contourf(x_el,y_el,counts_el_xy_new_red.T, levels=20)
cs=ax.pcolormesh(x_el,y_el,counts_el_xy_new_red.T, shading='gouraud')
# cs=ax.pcolormesh(x_el,y_el,counts_el_xy_new_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
ax.plot([0, 20], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, -33], [0, -19.5], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, 19], [0, 32], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.grid()

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([counts_el_xy_new_red.min(),counts_el_xy_new_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"MCP_el_xy_red.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"MCP_el_xy_red.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD

counts_rec_xy_new=np.ma.masked_where(np.sqrt(X_rec**2+Y_rec**2)<25,counts_rec_xy.reshape(-1)).reshape(len(x_rec),len(y_rec))
counts_rec_xy_new_red=np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5)
norm=mcolors.LogNorm(counts_rec_xy_new_red.min(),counts_rec_xy_new_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_rec.max()/y_rec.max())
# cs=ax.contourf(x_rec,y_rec,counts_rec_xy_new_red.T, levels=20)
# cs=ax.pcolormesh(x_rec,y_rec,counts_rec_xy_new_red.T, shading='gouraud')
cs=ax.pcolormesh(x_rec,y_rec,counts_rec_xy_new_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
ax.plot([0, 19], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, -33], [0, -19.5], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, 19], [0, 32], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.grid()

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([counts_el_xy_new_red.min(),counts_el_xy_new_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"MCP_rec_xy_red.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"MCP_rec_xy_red.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
counts_el_xy_red=np.ma.masked_array(counts_el_xy,counts_el_xy<1)
norm=mcolors.LogNorm(counts_el_xy_red.min(),counts_el_xy_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())

# cs=ax.contourf(x_el,y_el,counts_el_xy_red.T, levels=20)
# cs=ax.pcolormesh(x_el,y_el,counts_el_xy_red.T, shading='gouraud', norm=norm)
cs=ax.pcolormesh(x_el,y_el,counts_el_xy_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([counts_el_xy_new_red.min(),counts_el_xy_new_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"el_mom_ex.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_mom_ex.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
x_el_lin_r=fugi.remap(x_el_lin,-45,45)
y_el_lin_r=fugi.remap(y_el_lin,-45,45)
counts_el_lin_red=np.ma.masked_array(counts_el_lin,counts_el_lin<0.00001)
norm=mcolors.LogNorm(counts_el_lin_red.min(),counts_el_lin_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
# cs=ax.contourf(x_el_lin_r,y_el_lin_r,counts_el_lin_red, levels=40)
# cs=ax.pcolormesh(x_el_lin_r,y_el_lin_r,counts_el_lin_red, shading='gouraud')
cs=ax.pcolormesh(x_el_lin_r,y_el_lin_r,counts_el_lin_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([counts_el_lin_red.min(),counts_el_lin_red.max()])
# cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('deviation from linearity')

fig.savefig(dir+"MCP_el_linearity.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"MCP_el_linearity.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(tof1.max()/tof2.max())
counts_pipico_red=np.ma.masked_array(counts_pipico,counts_pipico<1)
# ax.set_title("fish fillet y-axis")

cs=ax.pcolormesh(tof1,tof2,counts_pipico_red.T, shading='gouraud')

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(0,5000)
ax.set_ylim(0,5000)
ax.set_aspect(aspect)

axins = ax.inset_axes([0.55, 0.06, 0.42, 0.42])
axins.pcolormesh(tof1,tof2,np.ma.masked_array(counts_pipico.T,counts_pipico.T<1), shading='gouraud')
axins.set_xlim(2800,3100)
axins.set_ylim(2900,3200)
axins.tick_params(axis='x', labelsize=12)
axins.tick_params(axis='y', labelsize=12)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([counts_pipico_red.min(),counts_pipico_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dirN2+"N2_pipico_insert.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"N2_pipico_insert.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(tmox_tof1.max()/tmox_tof2.max())
counts_pipico_tmox_red=np.ma.masked_array(counts_pipico_tmox,counts_pipico_tmox<2)
norm=mcolors.LogNorm(counts_pipico_tmox_red.min(),counts_pipico_tmox_red.max())

# ax.set_title("fish fillet y-axis")

cs=ax.pcolormesh(tmox_tof1,tmox_tof2,counts_pipico_tmox_red.T, shading='gouraud', norm=norm)

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(0,7500)
ax.set_ylim(0,7500)
ax.set_aspect(aspect)

rect1 = patches.Rectangle((2500, 6400), 3000, 500, linewidth=2, ls= "--", edgecolor='r', facecolor='none', alpha=0.4)
ax.add_patch(rect1)
rect2 = patches.Rectangle((2500, 5400), 3000, 500, linewidth=2, ls= ":", edgecolor='r', facecolor='none', alpha=0.4)
ax.add_patch(rect2)
rect3 = patches.Rectangle((2500, 4200), 2100, 500, linewidth=2, ls= "-.", edgecolor='r', facecolor='none', alpha=0.4)
ax.add_patch(rect3)
rect4 = patches.Rectangle((2650, 7000), 600, -3000, linewidth=2, ls= "--", edgecolor='b', facecolor='none', alpha=0.6)
ax.add_patch(rect4)
rect5 = patches.Rectangle((3800, 7000), 700, -3000, linewidth=2, ls= ":", edgecolor='b', facecolor='none', alpha=0.6)
ax.add_patch(rect5)
circ1 = patches.Circle((5150, 6650), radius=300, linewidth=2, ls= ":", edgecolor='b', facecolor='none', alpha=0.6)
ax.add_patch(circ1)

axins = ax.inset_axes([0., 0.65, 1., 1.])
axins.pcolormesh(tmox_tof1,tmox_tof2,np.ma.masked_array(counts_pipico_tmox.T,counts_pipico_tmox.T<2), shading='gouraud',norm=norm )
axins.set_xlim(2500,5500)
axins.set_ylim(6400,6900)
axins.tick_params(axis='x', labelsize=12)
axins.tick_params(axis='y', labelsize=12)
axins.set_aspect("equal")

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([counts_pipico_tmox_red.min(),counts_pipico_tmox_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"TMOX_pipico_insert.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"TMOX_pipico_insert.svg", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
couts_pipico_11_red=np.ma.masked_array(couts_pipico_11,couts_pipico_11<2)

cs=ax.pcolormesh(tof11_1,tof11_2,couts_pipico_11_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(3850,4500)
ax.set_ylim(6450,6800)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([couts_pipico_11_red.min(),couts_pipico_11_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"TFMOX_PIPICO_ch11.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
couts_pipico_9_red=np.ma.masked_array(couts_pipico_9,couts_pipico_9<2)
# norm=mcolors.LogNorm(couts_pipico_9_red.min(),couts_pipico_9_red.max())

cs=ax.pcolormesh(tof9_1,tof9_2,couts_pipico_9_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
# ax.set_xlim(3900,4500)
ax.set_ylim(6450,6800)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([couts_pipico_9_red.min(),couts_pipico_9_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"TFMOX_PIPICO_ch9.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12, 10))
couts_pipico_14_red=np.ma.masked_array(couts_pipico_14,couts_pipico_14<2)

cs=ax.pcolormesh(tof14_1,tof14_2,couts_pipico_14_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_xlim(4900,5400)
ax.set_ylim(6450,6800)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([couts_pipico_14_red.min(),couts_pipico_14_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"TFMOX_PIPICO_ch14.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
fig,ax= plt.subplots(1, figsize=(12.5, 10))

cs=ax.pcolormesh(ctheta,cKER,np.ma.masked_array(counts_KER.T,counts_KER.T<1), shading='gouraud')

divider = make_axes_locatable(ax)
ax_histy = divider.append_axes("left", 1.2, pad=0.15, sharey=ax)
ax_histy.plot(np.ma.masked_array(KER,KER<1),en, color=cmap_viridis(0), linewidth=3, alpha = 0.6)
ax_histy.invert_xaxis()

# ax.set_title("fish fillet y-axis")

ax.set_xlim([-1, 1])
ax.set_xlabel('cos\u03D1 [adm]')
ax_histy.set_ylim([1, 9])
ax_histy.set_ylabel('KER [eV]')
ax_histy.xaxis.set_tick_params(labelbottom=False)
ax_histy.grid()
ax.yaxis.set_tick_params(labelbottom=False)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"KER_gaus_ang.png", dpi=300, transparent=False,bbox_inches='tight')

In [ ]:
#3D MFPAD
counts_prel_red=np.ma.masked_array(counts_prel,counts_prel<1)
norm=mcolors.LogNorm(counts_prel_red.min(),counts_prel_red.max())

fig,ax= plt.subplots(1, figsize=(13.5, 8))

# cs=ax.contourf(x_el,y_el,counts_el_xy_red.T, levels=20)
# cs=ax.pcolormesh(prelx,prelz,counts_prel_red.T, shading='gouraud', norm=norm)
cs=ax.pcolormesh(prelx,prelz,counts_prel_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

circle1 = patches.Circle((0, 0), radius=110., fill=False, color="r", alpha=0.3)
ax.add_patch(circle1)

ax.set_xlabel('p$_{relx}$ [a.u.]')
ax.set_ylabel('p$_{relz}$ [a.u.]')
ax.set_ylim([None, 200])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([counts_prel_red.min(),counts_prel_red.max()])
# cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"p_rely_CH11.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_mom_ex.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

mod_count =np.clip(np.sum(counts,axis=1), -1, 1.5e4)

counts_ion_red=np.ma.masked_array(mod_count,mod_count<1)

my_cmap = mpl.cm.get_cmap('viridis').copy(); my_cmap.set_under('w')
cs,ax=fugi.plot_interpolation(pz_MF,px_MF,counts_ion_red.T, cmap=my_cmap, xstep=.01, ystep=.01, ax=ax, s_test=5, cont=False)

ax.set_xlabel('p$_{x}$ [a.u.]')
ax.set_ylabel('p$_{y}$ [a.u.]')
ax.set_xlim(-0.75,0.75)
ax.set_ylim(-1.8,1.5)

arrow1 = patches.Arrow(0, 0, 0.035, 1, width=0.05, color="r", alpha=0.6)
arrow2 = patches.Arrow(0, 0, -0.2, -0.05, width=0.09, color="r", alpha=0.6)
arrow3 = patches.Arrow(0, 0, 0.2, -0.95, width=0.05, color="r", alpha=0.6)
ax.add_patch(arrow1); ax.add_patch(arrow2); ax.add_patch(arrow3); 
text1=ax.text(-0.6,-0.1, "$OH^{0}$", rotation=180) #,fontweight="bold")
text2=ax.text(0.25,-1.35, "$C_{2}H_{i}^{+}$", rotation=180) #,fontweight="bold")
text3=ax.text(0.,1.2, "$CF_{3}^{+}$", rotation=180) #,fontweight="bold")
text1.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])
text2.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])
text3.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_clim()[0],cs.get_clim()[1]])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"MF_ion_CH11_flip.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

mod_count =np.clip(np.sum(counts,axis=1), -1, 1.5e4)

counts_ion_red=np.ma.masked_array(mod_count,mod_count<1)

my_cmap = mpl.cm.get_cmap('viridis').copy(); my_cmap.set_under('w')
cs,ax=fugi.plot_interpolation(pz_MF,px_MF,counts_ion_red.T, cmap=my_cmap, xstep=.01, ystep=.01, ax=ax, s_test=5, cont=False)

ax.set_xlabel('p$_{x}$ [a.u.]')
ax.set_ylabel('p$_{y}$ [a.u.]')
ax.set_xlim(-0.75,0.75)
ax.set_ylim(-1.8,1.5)

arrow1 = patches.Arrow(0, 0, 0.035, 1, width=0.05, color="r", alpha=0.6)
arrow2 = patches.Arrow(0, 0, -0.2, -0.05, width=0.09, color="r", alpha=0.6)
arrow3 = patches.Arrow(0, 0, 0.2, -0.95, width=0.05, color="r", alpha=0.6)
ax.add_patch(arrow1); ax.add_patch(arrow2); ax.add_patch(arrow3); 
text1=ax.text(-0.6,-0.1, "$OH^{0}$") #,fontweight="bold")
text2=ax.text(0.25,-1.35, "$C_{2}H_{i}^{+}$") #,fontweight="bold")
text3=ax.text(0.,1.2, "$CF_{3}^{+}$") #,fontweight="bold")
text1.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])
text2.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])
text3.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([cs.get_clim()[0],cs.get_clim()[1]])
cbar.set_ticklabels([0,"%d" % cs.get_array().max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"MF_ion_CH11.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

ID_local=str(IDs[1]).split("_")[0]
counts_newton12_red=np.ma.masked_array(newton12_all[0],newton12_all[0]<1)

cs=ax.pcolormesh(pz,px,counts_newton12_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('p$_{x}$ [a.u.]')
ax.set_ylabel('p$_{y}$ [a.u.]')
ax.set_xlim(-2,1.5)
ax.set_ylim(-1.5,1.5)

arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1, color="r", alpha=0.6)
ax.add_patch(arrow1)
text=ax.text(0.55,0.15, "$CH_{2}O^{+}$") #,fontweight="bold")
text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([counts_newton12_red.min(),counts_newton12_red.max()])
cbar.ax.set_ylabel('absolute counts')


fig.savefig(dir+"Netwon_"+ID_local+"_ch12.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
newton=[newton11_11_all[0],newton11_8_all[0],newton11_6_all[0],newton11_4_all[0],newton11_3_all[0]]
ID_local=str(IDs[0]).split("_")[0]
en=["11eV","8eV","6eV","4eV","3eV"]
text_en=["11,7 eV","8,1 eV","6,1 eV","4,1 eV","3,1 eV"]

for i,el in enumerate(newton):
        fig,ax= plt.subplots(1, figsize=(12, 10))
        counts_newton_red=np.ma.masked_array(el,el<2)
        cs=ax.pcolormesh(pz,px,counts_newton_red.T, shading='gouraud')
        # ax.set_title("fish fillet y-axis")

        ax.yaxis.set_tick_params(labelbottom=False)
        ax.xaxis.set_tick_params(labelbottom=False)

        cbar = fig.colorbar(cs,shrink=0.9, ax=ax, format='%.1e')
        cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
        # cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
        # cbar.set_ticks([counts_newton11_red.min(),counts_newton11_red.max()])
        cbar.ax.set_ylabel('absolute counts')
        
        divider = make_axes_locatable(ax)
        ax_histy = divider.append_axes("left", 1.2, pad=0.15, sharey=ax)
        ax_histy.plot(np.sum(counts_newton_red,axis=0),pz, color=cmap_viridis(0), linewidth=3, alpha = 0.6)
        ax_histy.invert_xaxis()
        ax_histy.set_ylabel('p$_{y}$ [a.u.]')
        ax_histy.set_ylim(-1.5,1.5)
        ax_histy.xaxis.set_tick_params(labelbottom=False)
        ax_histy.grid(which="minor")

        ax_histx = divider.append_axes("bottom", 1.2, pad=0.15, sharex=ax)
        ax_histx.plot(px,np.sum(counts_newton_red,axis=1), color=cmap_viridis(0), linewidth=3, alpha = 0.6)
        ax_histx.invert_yaxis()
        ax_histx.set_xlabel('p$_{x}$ [a.u.]')
        ax_histx.set_xlim(-2,1.5)
        ax_histx.yaxis.set_tick_params(labelbottom=False)
        ax_histx.grid()

        arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1, color="r", alpha=0.6)
        ax.add_patch(arrow1)
        if ID_local == "ID":
                text=ax.text(0.55,0.15, "$C_{2}H_{i=1,2,3}^{+}$") #,fontweight="bold")
        if ID_local == "ID2":
                text=ax.text(0.85,0.15, "$C_{2}H^{+}$") #,fontweight="bold")
        if ID_local == "ID3":
                text=ax.text(0.85,0.15, "$C_{2}H_{2}^{+}$") #,fontweight="bold")
        if ID_local == "ID4":
                text=ax.text(0.85,0.15, "$C_{2}H_{3}^{+}$") #,fontweight="bold")
        ax.text(-1.,1., "E$_{ph}$ = "+text_en[i])
                                 
        text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])


        fig.savefig(dir+"Netwon_"+ID_local+"_CH11_"+en[i]+".png", dpi=300, transparent=False, bbox_inches='tight')
        # fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

ID_local=str(IDs[0]).split("_")[0]
counts_newton11_red=np.ma.masked_array(newton11_8_all[0],newton11_8_all[0]<2)

cs=ax.pcolormesh(pz,px,counts_newton11_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('p$_{x}$ [a.u.]')
ax.set_ylabel('p$_{y}$ [a.u.]')
ax.set_xlim(-2,1.5)
ax.set_ylim(-1.5,1.5)

arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1, color="r", alpha=0.6)
ax.add_patch(arrow1)
if ID_local == "ID":
        text=ax.text(0.55,0.15, "$C_{2}H_{i=1,2,3}^{+}$") #,fontweight="bold")
if ID_local == "ID2":
        text=ax.text(0.85,0.15, "$C_{2}H^{+}$") #,fontweight="bold")
if ID_local == "ID3":
        text=ax.text(0.85,0.15, "$C_{2}H_{2}^{+}$") #,fontweight="bold")
if ID_local == "ID4":
        text=ax.text(0.85,0.15, "$C_{2}H_{3}^{+}$") #,fontweight="bold")
        ax.set_xlabel('p$_{x}$ [a.u.]')

text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# cbar.set_ticks([counts_newton11_red.min(),counts_newton11_red.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"Netwon_"+ID_local+"_ch11_8ev.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))

ID_local=str(IDs[0]).split("_")[0]
counts_newton9_red=np.ma.masked_array(newton9_all[0],newton9_all[0]<2)

counts_newton9_red=np.ma.masked_array(counts_newton9,counts_newton9<10)

cs=ax.pcolormesh(pz,px,counts_newton9_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('p$_{x}$ [a.u.]')
ax.set_ylabel('p$_{y}$ [a.u.]')
ax.set_xlim(-2,1.5)
ax.set_ylim(-1.5,1.5)

arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1,color="r", alpha=0.6)
ax.add_patch(arrow1)
if ID_local == "ID":
    text=ax.text(0.55,0.15, "$CH_{i=0,1,2,3}^{+}$") #,fontweight="bold")
if ID_local == "ID1":
    text=ax.text(0.85,0.15, "$CH^{+}$") #,fontweight="bold")
if ID_local == "ID2":
    text=ax.text(0.85,0.15, "$CH_{2}^{+}$") #,fontweight="bold")
if ID_local == "ID3":
    text=ax.text(0.85,0.15, "$CH_{3}^{+}$") #,fontweight="bold")

text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks(np.linspace(cs.get_array().min(),cs.get_array().max(),5))
# # cbar.set_ticks([counts_newton9_red.min(),counts_newton9_red.max()])
cbar.ax.set_ylabel('absolute counts')

fig.savefig(dir+"Netwon_"+ID_local+"_ch9.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
#sum el w-layer
fig,axes= plt.subplots(4,1, figsize=(7, 25), sharex=True, constrained_layout=True)

for i,ax in enumerate(axes.ravel()):
    # el=np.ma.masked_array(newton11_11_all[i+1],newton11_11_all[i+1]<5)
    el=np.ma.masked_array(newton11_11_all[i],newton11_11_all[i]<1)
    cs=ax.pcolormesh(pz,px,el.T, shading='gouraud')

    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.set_ylabel('p$_{y}$ [a.u.]')
    ax.set_xlim(-2,1.5)
    ax.set_ylim(-1.5,1.5)

    arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1,color="r", alpha=0.6)
    ax.add_patch(arrow1)

    if i == 0:
        text=ax.text(0.4,0.15, "$C_{2}H_{i=1,2,3}^{+}$") #,fontweight="bold")
    if i == 1:
        text=ax.text(0.85,0.15, "$C_{2}H^{+}$") #,fontweight="bold")
    if i == 2:
        text=ax.text(0.85,0.15, "$C_{2}H_{2}^{+}$") #,fontweight="bold")
    if i == 3:
        text=ax.text(0.85,0.15, "$C_{2}H_{3}^{+}$") #,fontweight="bold")
        ax.set_xlabel('p$_{x}$ [a.u.]')
    
    text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, location='top', ax=axes[0])
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([el.min(),el.max()])
# cbar.set_ticklabels(["min","max"])
# cbar.ax.set_ylabel('normalized counts', rotation="horizontal", ha='right')

fig.savefig(dir+"Netwon_all_CH11.png", dpi=300, transparent=False, bbox_inches='tight')

In [ ]:
#sum el w-layer
fig,axes= plt.subplots(4,1, figsize=(7, 25), sharex=True, constrained_layout=True)

for i,ax in enumerate(axes.ravel()):
    # el=np.ma.masked_array(newton9_all[i+1],newton9_all[i+1]<1)
    el=np.ma.masked_array(newton9_all[i],newton9_all[i]<1)
    cs=ax.pcolormesh(pz,px,el.T, shading='gouraud')

    ax.set_ylabel('p$_{y}$ [a.u.]')
    ax.set_xlim(-2,1.5)
    ax.set_ylim(-1.5,1.5)

    arrow1 = patches.Arrow(0, 0, 1, 0, width=0.1,color="r", alpha=0.6)
    ax.add_patch(arrow1)

    if i == 0:
        text=ax.text(0.4,0.15, "$CH_{i=0,1,2,3}^{+}$") #,fontweight="bold")
    if i == 1:
        text=ax.text(0.85,0.15, "$CH^{+}$") #,fontweight="bold")
    if i == 2:
        text=ax.text(0.85,0.15, "$CH_{2}^{+}$") #,fontweight="bold")
    if i == 3:
        text=ax.text(0.85,0.15, "$CH_{3}^{+}$") #,fontweight="bold")
        ax.set_xlabel('p$_{x}$ [a.u.]')
    
    text.set_path_effects([PathEffects.withStroke(linewidth=4, foreground='w', alpha=0.5)])

cbar = fig.colorbar(cs,shrink=0.9, location='top', ax=axes[0])
cbar.set_ticks([cs.get_array().min(),cs.get_array().max()])
# cbar.set_ticks([el.min(),el.max()])
# cbar.set_ticklabels(["min","max"])
# cbar.ax.set_ylabel('normalized counts', rotation="horizontal", ha='right')

fig.savefig(dir+"Netwon_all_CH9.png", dpi=300, transparent=False, bbox_inches='tight')
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)